In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
from cellpose import models, io, core
import pandas as pd
from numpy import isnan
import os.path

if core.use_gpu()==False: 
  #Warnings from the ZeroCost StarDist notebook
  print('You do not have GPU access.')
  print('Did you change your runtime ?') 
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')
  use_GPU=False
else:
  print("You have access to the GPU."+"\nDetails are:")
  print("*************************************************")
  use_GPU=True

In [ ]:
save_dir = "data/results/seg/"
img_dir = "data/images/"
base_dir = "data/"
model_cyto2 = models.Cellpose(gpu=use_GPU, model_type='cyto2')
df = pd.read_csv(base_dir+"muscle_atlas_2_7_filt_triple.csv")
img_list = df["Number"].tolist()
channel = [[0, 0]]

In [ ]:
for index, file in enumerate(img_list):
  print("Segmenting: ", file)
  if os.path.isfile(save_dir + "MASK_" + file + ".png"):
      print(file, " already segmented, passing...")
      continue
  img = skimage.io.imread(img_dir+file)
  # mask, flow, style, diam = model_cyto2.eval(img, diameter=calib_50um[index], channels=[[0,0]])
  mask, flow, style, diam = model_cyto2.eval(img, diameter=None, channels=channel)
  # save images in folder with the diameter value used in cellpose
  print("Segmentation complete . Saving Masks and flows")
  # Output name for masks
  mask_output_name = save_dir + "MASK_" + file + ".png"
  # Save mask as 16-bit in case this has to be used for detecting than 255 objects
  mask = mask.astype(np.uint16)
  plt.imsave(mask_output_name, mask)
  # Save Numpy Mask Array
  np.savez_compressed(save_dir+file+".npz", mask=mask, diam=diam, channel=channel)